In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
import tqdm
import os
from PIL import Image

2023-08-03 16:50:25.707401: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 16:50:26.712543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
RANDOM_SEED = 0
DATA_PATH = "../data/derived_data/items_with_floor/"
OUTPUT_PATH = "../data/derived_data/data_augmented/"

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [3]:
def random_crop(image):
    # Vertical and Horizontal coordinate of the top-left corner of the bounding box in image. 
    offset_height, offset_width = random.randint(0, image.shape[0]//2 - 100), random.randint(0, image.shape[1]//2 - 50)
    # Height and Width of the bounding box. 
    target_height, target_width = random.randint(image.shape[0]//2.5, image.shape[0] - offset_height), random.randint(image.shape[1]//2.5, image.shape[1] - offset_width)

    return tf.image.crop_to_bounding_box(image, offset_height, offset_width, target_height, target_width)

operations = [
    tf.keras.layers.RandomRotation(0.2, interpolation="nearest", seed=None),
    tf.keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=None),
    random_crop,
    tf.keras.layers.RandomHeight(0.25, interpolation='nearest', seed=None),
    tf.keras.layers.RandomWidth(0.25, interpolation='nearest', seed=None),
    tf.keras.layers.RandomContrast(0.25, seed=None),
    tf.keras.layers.RandomBrightness(0.5, value_range=(50, 200), seed=None),
    lambda image: tf.image.random_hue(image, 0.25, seed=None),
    lambda image: tf.image.random_saturation(image, 0, 1, seed=None),
]

In [4]:
N_RANDOM_OPERATIONS_COMBINATIONS = 100
MAX_OPERATIONS_COMBINED = 4
SAVE_IMAGE = True
IMG_SIZE = 128

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE, interpolation="nearest"),
  tf.keras.layers.Rescaling(1./255)
])

def no_agumentation(img_path):
    img = Image.open(img_path)
    img_arr = np.array(img)
    img_filename = img_path.split("/")[-1][:-4]
    os.makedirs(f"{OUTPUT_PATH}/{img_filename}", exist_ok=True)
    Image.fromarray(img_arr.astype(np.uint8), "RGB").save(f"{OUTPUT_PATH}/{img_filename}/{img_filename}_{str(i)}.png")

def augment_image(img_path):
    img = Image.open(img_path)
    img_arr = np.array(img)

    # without extension
    img_filename = img_path.split("/")[-1][:-4]

    for i, op in enumerate(operations):
        img_arr_augmented = op(img_arr)

        if SAVE_IMAGE:
            os.makedirs(f"{OUTPUT_PATH}/{img_filename}", exist_ok=True)
            Image.fromarray(img_arr_augmented.numpy().astype(np.uint8), "RGB").save(f"{OUTPUT_PATH}/{img_filename}/{img_filename}_{str(i)}.png")

        # img_arr_augmented = resize_and_rescale(img_arr_augmented)
        # np.save(f"{OUTPUT_PATH}/{img_filename}_{str(i)}", img_arr_augmented.numpy().astype(np.float16))

    random_ops_combs = [random.sample(operations, k=random.randint(1,MAX_OPERATIONS_COMBINED)) for _ in range(N_RANDOM_OPERATIONS_COMBINATIONS)]

    for i, random_ops in enumerate(random_ops_combs, start=i+1):
        img_arr_augmented = img_arr
        for op in random_ops:
            img_arr_augmented = op(img_arr_augmented)
        
        if SAVE_IMAGE:
            os.makedirs(f"{OUTPUT_PATH}/{img_filename}", exist_ok=True)
            Image.fromarray(img_arr_augmented.numpy().astype(np.uint8), "RGB").save(f"{OUTPUT_PATH}/{img_filename}/{img_filename}_{str(i)}.png")

        # img_arr_augmented = resize_and_rescale(img_arr_augmented)
        # np.save(f"{OUTPUT_PATH}/{img_filename}_{str(i)}", img_arr_augmented.numpy().astype(np.float16))

2023-08-03 16:50:52.692779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 16:50:52.876070: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 16:50:52.876113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 16:50:52.880760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 16:50:52.880839: I tensorflow/compile

In [6]:
for img_path in tqdm.tqdm(glob.glob(DATA_PATH + "*")):
    augment_image(img_path)
    # no_agumentation(img_path)

100%|██████████| 716/716 [20:05<00:00,  1.68s/it]
